In [1]:
!pip install pysqlite3

In [14]:
import pymysql
from pysqlite3 import dbapi2 as sqlite3
import pandas as pd
from IPython.display import display, HTML

In [15]:
print(sqlite3.sqlite_version_info)

(3, 39, 3)


In [26]:
def display_query_result(cursor, query):
    cursor.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    styled_html = f"<b><font color='green'>{query}</font></b><br>{df.to_html(index=False)}"
    display(HTML(styled_html))

In [25]:
host = "127.0.0.1"
user = "root"
password = "Sree123@"


connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

cursor = connection.cursor()

In [18]:
cursor.execute("CREATE DATABASE IF NOT EXISTS music_streaming_1;")
print("Database 'music_streaming_1' has been created .")

Database 'music_streaming_1' has been created .


In [28]:
cursor.execute("use music_streaming_1 ")
print("Database 'music_streaming_1' has been selected")

Database 'music_streaming_1' has been selected


In [20]:
cursor.execute("""
      CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE
      );
""")
print("Users Table has been created")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Songs (
    song_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    artist VARCHAR(100) NOT NULL,
    genre VARCHAR(100)
  );
"""
)
print("Songs Table has been created")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Listens (
      listen_id INTEGER PRIMARY KEY,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      rating FLOAT,
      listen_time TIMESTAMP,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
    );
""")
print("Listens Table has been created")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Recommendations (
      recommendation_id INTEGER NOT NULL,
      recommendation_time TIMESTAMP,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
  );
""")
print("Recommendations Table has been created")

Users Table has been created
Songs Table has been created
Listens Table has been created
Recommendations Table has been created


In [21]:
cursor.execute("""
  INSERT INTO Users (user_id, name, email)
  VALUES
    (1, 'Mickey', 'mickey@example.com'),
    (2, 'Minnie', 'minnie@example.com'),
    (3, 'Daffy', 'daffy@example.com'),
    (4, 'Pluto', 'pluto@example.com');
""")
print("User Data has been inserted")

cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
    (1, 'Evermore', 'Taylor Swift', 'Pop'),
    (2, 'Willow', 'Taylor Swift', 'Pop'),
    (3, 'Shape of You', 'Ed Sheeran', 'Rock'),
    (4, 'Photograph', 'Ed Sheeran', 'Rock'),
    (5, 'Shivers', 'Ed Sheeran', 'Rock'),
    (6, 'Yesterday', 'Beatles', 'Classic'),
    (7, 'Yellow Submarine', 'Beatles', 'Classic'),
    (8, 'Hey Jude', 'Beatles', 'Classic'),
    (9, 'Bad Blood', 'Taylor Swift', 'Rock'),
    (10, 'DJ Mix', 'DJ', NULL);
""")
print("Songs Data has been inserted")

cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
    (1, 1, 1, 4.5, '2024-08-30 14:35:00'),
    (2, 1, 2, 4.2, NULL),
    (3, 1, 6, 3.9, '2024-08-29 10:15:00'),
    (4, 2, 2, 4.7, NULL),
    (5, 2, 7, 4.6, '2024-08-28 09:20:00'),
    (6, 2, 8, 3.9, '2024-08-27 16:45:00'),
    (7, 3, 1, 2.9, NULL),
    (8, 3, 2, 4.9, '2024-08-26 12:30:00'),
    (9, 3, 6, NULL, NULL);
""")
print("Listens Data has been inserted")

User Data has been inserted
Songs Data has been inserted
Listens Data has been inserted


In [22]:
connection.commit()
print("All changes committed to the database.")

All changes committed to the database.


In [11]:
query = "SELECT * FROM Users;"
display_query_result(cursor,query)

user_id,name,email
1,Mickey,mickey@example.com
2,Minnie,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


In [12]:
query = "SELECT * FROM Songs;"
display_query_result(cursor, query)

song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


In [90]:
query = "SELECT * FROM Listens;"
display_query_result(cursor,query)

listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,NaT
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,NaT
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,NaT
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,NaT


In [91]:
qry_classic_songs = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic';
"""
display_query_result(cursor, qry_classic_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


In [67]:
#Find titles and artists if songs in Classic Genre with titles starting like Ye...

qry_classic_songs2 = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic' AND
  Songs.title LIKE 'Ye%';
"""
display_query_result(cursor, qry_classic_songs2)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles


In [68]:
#List all the genres in the Songs table
qry_genres = """
SELECT genre from Songs;
"""
display_query_result(cursor, qry_genres)

genre
Pop
Pop
Rock
Rock
Rock
Classic
Classic
Classic
Rock
None


In [69]:
#Find the unique genres from the Songs table

qry_distinct = """
SELECT DISTINCT genre
FROM Songs;
"""

display_query_result(cursor, qry_distinct)

genre
Pop
Rock
Classic
None


In [70]:
#Find the number of songs by Taylor Swift in different genres

qry_taylor_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
WHERE Songs.artist='Taylor Swift'
GROUP BY artist, genre;
"""
display_query_result(cursor, qry_taylor_count)

artist,genre,num_songs
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [71]:
#Find the number of songs by all artists in different genres

qry_artist_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
GROUP BY artist, genre;
"""
display_query_result(cursor, qry_artist_count)


artist,genre,num_songs
Taylor Swift,Pop,2
Ed Sheeran,Rock,3
Beatles,Classic,3
Taylor Swift,Rock,1
DJ,None,1


In [72]:
#Create one single table for songs and listens. Redundancies are allowed

qry_large_table = """
SELECT * FROM Songs
LEFT JOIN Listens ON Songs.song_id=Listens.song_id
LEFT JOIN Users ON Listens.user_id=Users.user_id;
"""
display_query_result(cursor,qry_large_table)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
1,Evermore,Taylor Swift,Pop,1.0,1.0,1,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,Evermore,Taylor Swift,Pop,7.0,3.0,1,2.9,NaT,3.0,Daffy,daffy@example.com
2,Willow,Taylor Swift,Pop,2.0,1.0,2,4.2,NaT,1.0,Mickey,mickey@example.com
2,Willow,Taylor Swift,Pop,4.0,2.0,2,4.7,NaT,2.0,Minnie,minnie@example.com
2,Willow,Taylor Swift,Pop,8.0,3.0,2,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,3,NaN,NaT,NaN,None,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,4,NaN,NaT,NaN,None,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,5,NaN,NaT,NaN,None,None
6,Yesterday,Beatles,Classic,3.0,1.0,6,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com
6,Yesterday,Beatles,Classic,9.0,3.0,6,NaN,NaT,3.0,Daffy,daffy@example.com


In [73]:
#Find the Highly Rates Songs. Consider Rating above 4.6 as highly rated

qry_rating_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, Listens.rating
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
WHERE Listens.rating > 4.6
"""
display_query_result(cursor,qry_rating_songs)

#Find tha average rating of every song

qry_avg_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating)
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, Songs.title, Songs.artist
"""
display_query_result(cursor,qry_avg_songs)



song_id,title,artist,rating
2,Willow,Taylor Swift,4.7
2,Willow,Taylor Swift,4.9


song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,3.7
2,Willow,Taylor Swift,4.6
6,Yesterday,Beatles,3.9
7,Yellow Submarine,Beatles,4.6
8,Hey Jude,Beatles,3.9


In [74]:
#Find the popular songs by counting the listens

qry_pop_songs = """
SELECT Songs.song_id, Songs.title, Songs.artist, count(Listens.song_id) as Popular_Songs_Count
FROM Songs
JOIN Listens
ON Songs.song_id=Listens.song_id
GROUP BY Songs.title, Songs.artist, Songs.song_id
ORDER BY COUNT(Listens.song_id) DESC;
"""

display_query_result(cursor, qry_pop_songs)

song_id,title,artist,Popular_Songs_Count
2,Willow,Taylor Swift,3
1,Evermore,Taylor Swift,2
6,Yesterday,Beatles,2
7,Yellow Submarine,Beatles,1
8,Hey Jude,Beatles,1


In [75]:
#Find songs by Ed Sheeran and Taylor Swift

qry_edtaylor="""
SELECT title, artist
FROM Songs
WHERE artist IN ('Ed Sheeran', 'Taylor Swift');
"""
display_query_result(cursor, qry_edtaylor)

#Find songs from both pop and rock genres

qry_popandrock = """
SELECT title, artist
FROM Songs
WHERE genre='Pop'
UNION
SELECT title, artist
FROM Songs
WHERE genre='Rock'
"""
display_query_result(cursor, qry_popandrock)


title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Shape of You,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran
Bad Blood,Taylor Swift


title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Shape of You,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran
Bad Blood,Taylor Swift


In [76]:
#Find songs listened to by user_id 1

qry_listen_usr="""
SELECT title, artist
FROM Songs
WHERE song_id IN (SELECT song_id FROM Listens WHERE Listens.listen_time IS NULL);
"""
display_query_result(cursor,qry_listen_usr)

title,artist
Willow,Taylor Swift
Evermore,Taylor Swift
Yesterday,Beatles


In [23]:
#Data Insertion into Reccomandation table 
query=cursor.execute(""" INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id, song_id)
SELECT 
  ROW_NUMBER() OVER () AS recommendation_id,
  CURRENT_TIMESTAMP() AS recommendation_time,
  user_id,
  song_id
FROM (
  WITH song_similarity AS (
    SELECT u1.song_id AS song1, u2.song_id AS song2, COUNT(*) AS common_users
    FROM Listens u1
    JOIN Listens u2 ON u1.user_id = u2.user_id AND u1.song_id != u2.song_id
    GROUP BY u1.song_id, u2.song_id
    HAVING COUNT(*) > 1
  ),
  recs AS (
    SELECT L.user_id, song2 AS song_id
    FROM song_similarity
    JOIN Listens AS L ON L.song_id = song_similarity.song1
    WHERE song_similarity.song2 NOT IN (
      SELECT song_id FROM Listens AS temp WHERE temp.user_id = L.user_id
    )
  )
  SELECT * FROM recs
) AS recommendations;
""")



In [77]:
#Computing the recommendations
#Find the song pairs which are shared across > 1 user

qry_sharing="""
WITH song_similarity AS (
SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
FROM LISTENS u1
JOIN LISTENS u2
ON u1.user_id=u2.user_id
AND u1.song_id != u2.song_id
GROUP BY u1.song_id, u2.song_id
HAVING COUNT(*)>1
),

recs AS (
  SELECT user_id, song2 as song_id
  FROM song_similarity
  JOIN Listens as L
  ON L.song_id = song_similarity.song1
  WHERE song_similarity.song2 NOT IN
  (SELECT song_id FROM Listens as temp where temp.user_id=L.user_id)
)
select * from recs;
"""
display_query_result(cursor, qry_sharing)

user_id,song_id
2,1
2,6


In [78]:
# Recommendations for minnie

query = '''SELECT s.song_id, s.title, s.artist, AVG(l.rating) AS avg_rating
FROM Songs s
JOIN Listens l ON s.song_id = l.song_id
WHERE s.song_id NOT IN (
    SELECT song_id FROM Listens WHERE user_id = 2
)
AND l.rating IS NOT NULL
GROUP BY s.song_id, s.title, s.artist
ORDER BY avg_rating DESC;
'''

display_query_result(cursor, query)

song_id,title,artist,avg_rating
6,Yesterday,Beatles,3.9
1,Evermore,Taylor Swift,3.7


In [79]:
# Recommendations on the basis of listen time

query = '''SELECT s.song_id, s.title, s.artist, AVG(l.rating) AS avg_rating
FROM Songs s
JOIN Listens l ON s.song_id = l.song_id
WHERE l.listen_time IS NOT NULL
AND l.rating IS NOT NULL
AND s.song_id NOT IN (
    SELECT song_id FROM Listens WHERE user_id = 2
)
GROUP BY s.song_id, s.title, s.artist
ORDER BY avg_rating DESC;

'''

display_query_result(cursor, query)

song_id,title,artist,avg_rating
1,Evermore,Taylor Swift,4.5
6,Yesterday,Beatles,3.9


In [92]:
# Generating new recommendations

query = '''SELECT s.song_id, s.title, s.artist, AVG(l.rating) AS avg_rating
FROM Listens l
JOIN Songs s ON l.song_id = s.song_id
WHERE l.user_id != 2
AND l.song_id NOT IN (
    SELECT song_id FROM Listens WHERE user_id = 2
)
AND l.rating IS NOT NULL
GROUP BY s.song_id, s.title, s.artist
ORDER BY avg_rating DESC;
'''

display_query_result(cursor, query)

song_id,title,artist,avg_rating
6,Yesterday,Beatles,3.9
1,Evermore,Taylor Swift,3.7


In [24]:
connection.commit()
connection.close()